**Process类**  
Process([group [, target [, name [, args [, kwargs]]]]])，由该类实例化得到的对象，表示一个子进程中的任务（尚未启动）

强调：
1. 需要使用关键字的方式来指定参数
2. args指定的为传给target函数的位置参数，是一个元组形式，必须有逗号

group参数未使用，值始终为None

target表示调用对象，即子进程要执行的任务

args表示调用对象的位置参数元组，args=(1,2,'hexin',)

kwargs表示调用对象的字典,kwargs={'name':'hexin','age':18}

name为子进程的名称  

**方法介绍**
p.start()：启动进程，并调用该子进程中的p.run() 
p.run():进程启动时运行的方法，正是它去调用target指定的函数，我们自定义类的类中一定要实现该方法  

p.terminate():强制终止进程p，不会进行任何清理操作，如果p创建了子进程，该子进程就成了僵尸进程，使用该方法需要特别小心这种情况。如果p还保存了一个锁那么也将不会被释放，进而导致死锁
p.is_alive():如果p仍然运行，返回True

p.join([timeout]):主线程等待p终止（强调：是主线程处于等的状态，而p是处于运行的状态）。timeout是可选的超时时间，需要强调的是，p.join只能join住start开启的进程，而不能join住run开启的进程

**属性介绍**  
p.daemon：默认值为False，如果设为True，代表p为后台运行的守护进程，当p的父进程终止时，p也随之终止，并且设定为True后，p不能创建自己的新进程，必须在p.start()之前设置

p.name:进程的名称

p.pid：进程的pid

p.exitcode:进程在运行时为None、如果为–N，表示被信号N结束(了解即可)

p.authkey:进程的身份验证键,默认是由os.urandom()随机生成的32字符的字符串。这个键的用途是为涉及网络连接的底层进程间通信提供安全性，这类连接只有在具有相同的身份验证键时才能成功（了解即可）

In [1]:
import time
import random
from multiprocessing import Process

创建并开启子进程  
**方法一**

In [15]:
def test(name):
    print("%s run" % name)
    time.sleep(random.randrange(0,2))
    print("%s end" % name)

p_obj = []
for i in range(0,4):
    p = Process(target=test, args=('%s' % i,))
    p_obj.append(p)
    p.start()
print('end')

0 run
1 run
end
2 run
3 run
3 end
0 end
1 end
2 end


**方法二**

In [24]:
class Test(Process):
    def __init__(self,name):
        super().__init__()
        self.name=name
        
    def run(self):
        print('%s run' % self.name)
        time.sleep(random.randrange(0,2))
        print('%s end' % self.name)
    
p_obj = []
for i in range(0,5):
    p = Test('%s' % i)
    p_obj.append(p)
    p.start()
print('end')

0 run
1 run
2 run
2 end
end
3 run
3 end
4 run
4 end
0 end
1 end


**run()**和**start()**的区别在于  
start()方法

开始线程活动。

对每一个线程对象来说它只能被调用一次，它安排对象在一个另外的单独线程中调用run()方法（而非当前所处线程）。

当该方法在同一个线程对象中被调用超过一次时，会引入RuntimeError(运行时错误)。

run()方法

代表了线程活动的方法。

你可以在子类中重写此方法。标准run()方法调用了传递给对象的构造函数的可调对象作为目标参数，如果有这样的参数的话，顺序和关键字参数分别从args和kargs取得。



In [32]:

import threading
 
class myThread(threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
 
    def run(self):
        currentTreadname = threading.currentThread()
        print("running in", currentTreadname)
        
thread = myThread(1,"mythrd",1)
thread.run()
thread.start()

running in <_MainThread(MainThread, started 4578622784)>
running in <myThread(mythrd, started 123145352839168)>


In [34]:
from multiprocessing import Process
import time
import random

class Piao(Process):
    def __init__(self,name):
        self.name=name
        super().__init__()
    def run(self):
        print('%s is piaoing' %self.name)
        time.sleep(random.randrange(1,3))
        print('%s is piao end' %self.name)


p=Piao('e')
p.daemon=True #一定要在p.start()前设置,设置p为守护进程,禁止p创建子进程,并且父进程死,p跟着一起死
p.start()
p.join() #等待p停止,等0.0001秒就不再等了
print('开始')

Piao-30 is piaoing
Piao-30 is piao end
开始


每启动一个新的进程相当创建全局变量的一份副本，子进程里的数据修改无法影响到主进程以及其他子进程中的数据，不同子进程之间的数据也不能共享，这是多进程与多线程最明显的区别  
所以下面都能成功

In [75]:
from multiprocessing import Process, Lock
import time
import random
import os

def work(lock):
    lock.acquire()
    global a
    if a > 0:
        print('%s success' % os.getpid(),a)
        a -= 1
        time.sleep(random.randrange(0,2))
    else:
        print('%s fail' % os.getpid())
    lock.release()

if __name__ == '__main__':
    a = 2
    lock = Lock()
    p_obj =[]
    for i in range(0,5):
        p = Process(target=work,args=(lock,))
        p.start()
        p_obj.append(p)


12883 success 2
12884 success 2
12885 success 2
12886 success 2
12887 success 2


多进程在读写文件的时候 加锁

In [79]:
from multiprocessing import Process,Lock
import json
import time
import random
import os

def work(filename,lock): #买票
    lock.acquire()
#     with lock:
    with open(filename,encoding='utf-8') as f:
        dic=json.loads(f.read())
        # print('剩余票数: %s' % dic['count'])
    if dic['count'] > 0:
        dic['count']-=1
        time.sleep(random.randint(1,2)) #模拟网络延迟
        with open(filename,'w',encoding='utf-8') as f:
            f.write(json.dumps(dic))
        print('%s 购票成功' %os.getpid())
    else:
        print('%s 购票失败' %os.getpid())
    lock.release()

if __name__ == '__main__':
    lock=Lock()
    p_l=[]
    for i in range(10):
        p=Process(target=work,args=('db.txt',lock))
        p_l.append(p)
        p.start()
    for p in p_l:
        p.join()

    print('主线程')

12937 购票成功
12938 购票成功
12939 购票成功
12940 购票失败
12941 购票失败
12942 购票失败
12943 购票失败
12944 购票失败
12945 购票失败
12946 购票失败
主线程


-1